# Time for Search

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

## Connect to Weaviate

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
        "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

In [ ]:
# import weaviate, os

# Connect with Weaviate Embedded
# client = weaviate.connect_to_embedded(
#     version="1.24.14",

#     headers={
#         "X-Google-Api-Key": AI_STUDIO_KEY,
#     })

# Connect to a Weaviate Cloud instance
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

#     headers={
#        "X-Palm-Api-Key": AI_STUDIO_KEY,
#     }
# )

# client.is_ready()

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
# client.query.

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
# client.query.

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
# client.query.

## Retrieval Augmented Generation!!!
[Docs - RAG](https://weaviate.io/developers/weaviate/search/generative)

In [ ]:
# client.generate